In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
%%cuda

UsageError: %%cuda is a cell magic, but the cell body is empty.


In [ ]:
%%writefile vector_add.cu
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void add(const float *a, const float *b, float *c, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    for(; idx < size; idx += blockDim.x * gridDim.x) {
        c[idx] = a[idx] + b[idx];
    }
}

int main() {
    const int N = 1024;
    size_t size = N * sizeof(float);

    float *h_a = (float *)malloc(size);
    float *h_b = (float *)malloc(size);
    float *h_c = (float *)malloc(size);

    for (int i = 0; i < N; i++) {
        h_a[i] = rand() / (float)RAND_MAX;
        h_b[i] = rand() / (float)RAND_MAX;
    }

    float *d_a, *d_b, *d_c;
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_c, size);

    cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice);

    int threadsPerBlock = 256;
    int blocks = (N + threadsPerBlock - 1) / threadsPerBlock;

    add<<<blocks, threadsPerBlock>>>(d_a, d_b, d_c, N);

    cudaDeviceSynchronize();

    cudaMemcpy(h_c, d_c, size, cudaMemcpyDeviceToHost);

    for (int i = 0; i < 5; i++) {
        printf("%f + %f = %f\n", h_a[i], h_b[i], h_c[i]);
    }

    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(h_a);
    free(h_b);
    free(h_c);
}


Writing vector_add.cu


In [ ]:
!nvcc -arch=sm_75 vector_add.cu -o vector_add

In [ ]:
!./vector_add

0.840188 + 0.394383 = 1.234571
0.783099 + 0.798440 = 1.581539
0.911647 + 0.197551 = 1.109199
0.335223 + 0.768230 = 1.103452
0.277775 + 0.553970 = 0.831745
